In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import esmtools as et
from esmtools import stats

import shapefile
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gp
import regionmask
import matplotlib.patches as mpatches

In [2]:
# Open dataset for co3satarag
dir = '/home/jovyan/SOARS_2021/Data/CESM2/'
fn_co3satarag = 'co3satarag_Omon_CESM2_omip2_r1i1p1f1_gr_030601-036612.nc'

# Open dataset for co3
#dir = '/home/jovyan/SOARS_2021/Data/CESM2/'
fn_co3 = 'co3_Omon_CESM2_omip2_r1i1p1f1_gr_030601-036612.nc'

data_co3satarag = xr.open_dataset(dir+fn_co3satarag)
data_co3 = xr.open_dataset(dir+fn_co3)

/home/jovyan/my-conda-envs/myenv/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'co3satarag' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/jovyan/my-conda-envs/myenv/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'co3' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [3]:
co3satarag = data_co3satarag['co3satarag']
co3 = data_co3['co3']

co3satarag

<xarray.DataArray 'co3satarag' (time: 732, lev: 33, lat: 180, lon: 360)>
[1565308800 values with dtype=float32]
Coordinates:
  * time     (time) object 0306-01-15 12:00:00 ... 0366-12-15 12:00:00
  * lev      (lev) float64 0.0 10.0 20.0 30.0 ... 4e+03 4.5e+03 5e+03 5.5e+03
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
Attributes: (12/19)
    cell_measures:  area: areacello volume: volcello
    cell_methods:   area: mean where sea time: mean
    comment:        Model data on the 1x1 grid includes values in all cells f...
    description:    Mole concentration (number of moles per unit volume: mola...
    frequency:      mon
    id:             co3satarag
    ...             ...
    time_label:     time-mean
    time_title:     Temporal mean
    title:          Mole Concentration of Carbonate Ion in Equilibrium with P...
    type:           real
    units:          mol m-3
    variable_id:    co3satarag

In [ ]:
data_omg_a = co3 / co3satarag

In [ ]:
# Change the time to be from DatetimeNoLeap to a Datetime64
data_omg_a['time'] = np.arange('1958-01-01','2019-01-01',dtype = 'datetime64[M]')
data_omg_a

In [ ]:
# select dimensions!
data_omg_a['time'] = pd.date_range("1958-01", "2018-12", freq="MS")
omg_a = data_omg_a['co3satarag']
omg_a = omg_a.sel(lon=slice(190,270),lat=slice(-10,70), lev=0)

In [ ]:
# remove long term trend for each grid cell
#var = stats.rm_poly(omg_a,order=2,dim='time',nan_policy='drop')
var = omg_a

In [ ]:
# 1982-2010 period
base = var.sel(time=slice('1982-01','2010-12'))
tos_base = base

In [ ]:
# Blob period
blob = var.sel(time=slice('2013-07','2016-06'))
tos_blob = blob

In [ ]:
# Difference between the two periods
blob_anom = blob.groupby('time.month') - base.groupby('time.month').mean()

In [ ]:
# Resample to two month resolution
cesm = blob_anom.resample(time='2M').mean()

In [ ]:
def panels(data,description,variable,depth,lower_bound,upper_bound, step,cmap):
    f, axs = plt.subplots(nrows=3,ncols=6,figsize=(20,6),
                          subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180))) 
    axs = axs.flatten()
    for i in range(0,18):
        im = axs[i].contourf(cesm.lon, cesm.lat, cesm.isel(time=i),
                         transform=ccrs.PlateCarree(), 
                         cmap=cmap,
                         levels=np.arange(lower_bound, upper_bound, step),
                         extend='both')
        axs[i].add_feature(cfeature.LAND, color='k')
        axs[i].add_patch(mpatches.Rectangle(xy=[210, 40], width=15, height=10,
                                    edgecolor='k',
                                    facecolor='none',
                                    transform=ccrs.PlateCarree())
                 )
    plt.suptitle("CESM2: " + description + " " + variable + " " + depth) # + " (de-trended)")
    axs[0].set_title("Jul-Aug"); axs[1].set_title("Sept-Oct")
    axs[2].set_title("Nov-Dec"); axs[3].set_title("Jan-Feb")
    axs[4].set_title("Mar-Apr"); axs[5].set_title("May-Jun")
    plt.text(-790,215,'2013-14')
    plt.text(-790,120,'2014-15')
    plt.text(-790,25,'2015-16')
    f.subplots_adjust(right=0.8)
    cbar_ax = f.add_axes([0.82, 0.15, 0.05, 0.7])
    f.colorbar(im, cax=cbar_ax,fraction=0.046,pad=0.04)
    
panels(omg_a,'Omega Aragonite','NE','Pacific',-0.001,0.001, 0.0001,'bwr_r')